In [5]:
import pandas as pd
import sqlite3

# Exercise 02 : Join

## create a connection to the database using the library sqlite3

In [6]:
con = sqlite3.connect('data/checking-logs.sqlite')
con

## create a new table datamart in the database by joining the tables pageviews and checker using only one query
- the table should have the following columns: uid, labname, first_commit_ts, first_view_ts
- first_commit_ts is just a new name of the column timestamp from the checker table, it shows the first commit - from a particular lab and from a particular user
- first_view_ts is the first visit of a user to the table pageviews, timestamp when a user visited the newsfeed
- status = ’ready’ should still be a filter
- numTrials = 1 should still be a filter
- labnames should still be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’
- the table should contain only the users (uids with user_*) and not the admins
- first_commit_ts and first_view_ts should be parsed as datetime64[ns]

In [8]:
create_table_query = """
CREATE TABLE datamart AS
SELECT 
    checker.uid,
    checker.labname,
    DATETIME(checker.timestamp) AS first_commit_ts,
    DATETIME(pageviews.datetime) AS first_view_ts
FROM 
    checker
LEFT JOIN 
     pageviews
ON 
    pageviews.uid = checker.uid
WHERE
    checker.status = 'ready' 
AND 
    checker.numTrials = 1
AND 
    checker.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
AND
    checker.uid LIKE 'user_%';
"""
con.execute(create_table_query)
con.commit()

In [9]:
datamart = pd.io.sql.read_sql("SELECT * FROM datamart;", con)
datamart['first_commit_ts'] = pd.to_datetime(datamart['first_commit_ts'])
datamart['first_view_ts'] = pd.to_datetime(datamart['first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02,NaT
1,user_4,laba04,2020-04-17 11:33:17,NaT
2,user_4,laba04s,2020-04-17 11:48:41,NaT
3,user_17,project1,2020-04-18 07:56:45,2020-04-18 10:56:55
4,user_17,project1,2020-04-18 07:56:45,2020-04-18 12:05:48
...,...,...,...,...
5659,user_17,laba06s,2020-05-21 17:39:17,2020-05-15 13:01:22
5660,user_17,laba06s,2020-05-21 17:39:17,2020-05-17 17:39:50
5661,user_17,laba06s,2020-05-21 17:39:17,2020-05-18 20:47:36
5662,user_17,laba06s,2020-05-21 17:39:17,2020-05-18 22:05:00


## using Pandas methods, create two dataframes: test and control
- test should have the users that have the values in first_view_ts
- control should have the users that have missing values in first_view_ts
- replace the missing values in the control with the average first_view_ts of the test users, we will use this value for the future analysis
- save both tables into the database, you will use them in the next exercises

In [10]:
test = datamart[datamart['first_view_ts'].notnull()]
test

,uid,labname,first_commit_ts,first_view_ts
3,user_17,project1,2020-04-18 07:56:45,2020-04-18 10:56:55
4,user_17,project1,2020-04-18 07:56:45,2020-04-18 12:05:48
5,user_17,project1,2020-04-18 07:56:45,2020-04-18 12:06:13
6,user_17,project1,2020-04-18 07:56:45,2020-04-18 12:06:35
7,user_17,project1,2020-04-18 07:56:45,2020-04-21 19:04:25
...,...,...,...,...
5659,user_17,laba06s,2020-05-21 17:39:17,2020-05-15 13:01:22
5660,user_17,laba06s,2020-05-21 17:39:17,2020-05-17 17:39:50
5661,user_17,laba06s,2020-05-21 17:39:17,2020-05-18 20:47:36
5662,user_17,laba06s,2020-05-21 17:39:17,2020-05-18 22:05:00


In [11]:
control = datamart[datamart['first_view_ts'].isnull()]
control

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02,NaT
1,user_4,laba04,2020-04-17 11:33:17,NaT
2,user_4,laba04s,2020-04-17 11:48:41,NaT
53,user_2,laba04,2020-04-18 13:42:35,NaT
54,user_2,laba04s,2020-04-18 13:51:22,NaT
...,...,...,...,...
5091,user_2,laba06s,2020-05-19 14:45:03,NaT
5401,user_6,laba06s,2020-05-20 14:50:07,NaT
5551,user_7,laba06s,2020-05-20 23:05:37,NaT
5552,user_23,laba06,2020-05-21 08:34:10,NaT


In [12]:
control.loc[:,'first_view_ts'] = control['first_view_ts'].fillna(value=test['first_view_ts'].mean())
control

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02,2020-05-11 21:58:26.509582848
1,user_4,laba04,2020-04-17 11:33:17,2020-05-11 21:58:26.509582848
2,user_4,laba04s,2020-04-17 11:48:41,2020-05-11 21:58:26.509582848
53,user_2,laba04,2020-04-18 13:42:35,2020-05-11 21:58:26.509582848
54,user_2,laba04s,2020-04-18 13:51:22,2020-05-11 21:58:26.509582848
...,...,...,...,...
5091,user_2,laba06s,2020-05-19 14:45:03,2020-05-11 21:58:26.509582848
5401,user_6,laba06s,2020-05-20 14:50:07,2020-05-11 21:58:26.509582848
5551,user_7,laba06s,2020-05-20 23:05:37,2020-05-11 21:58:26.509582848
5552,user_23,laba06,2020-05-21 08:34:10,2020-05-11 21:58:26.509582848


In [13]:
test.to_sql('test', con, if_exists='replace', index=False)
control.to_sql('control', con, if_exists='replace', index=False)

81

## close the connection

In [14]:
con.close()